# ELO Rating System for Hockey Game Prediction

This notebook demonstrates how to use the ELO rating system to predict hockey game outcomes. The ELO system, originally developed for chess, has been adapted for team sports with additional adjustments for:

- Home ice advantage
- Rest and fatigue (back-to-back penalties)
- Travel distance
- Injury impacts
- Margin of victory
- Division-based initialization

## Table of Contents

1. Setup and Imports
2. Understanding the ELO Formula
3. Loading and Preparing Data
4. Training the Model
5. Evaluating Performance
6. Making Predictions
7. Hyperparameter Tuning

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Configure plotting
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

## 2. The ELO Model Class

The ELO rating system works by:
1. Assigning each team an initial rating (default: 1500)
2. After each game, updating ratings based on the expected vs actual outcome
3. Teams gain rating points for wins and lose points for losses
4. The magnitude of change depends on the K-factor and margin of victory

### Key Formulas

**Expected Score:**
$$E_A = \frac{1}{1 + 10^{(R_B - R_A) / 400}}$$

**Rating Update:**
$$R'_A = R_A + K \times (S_A - E_A)$$

Where:
- $R_A$, $R_B$ = Current ratings of teams A and B
- $E_A$ = Expected score (win probability) for team A
- $S_A$ = Actual score (1 for win, 0 for loss)
- $K$ = K-factor controlling rating volatility

In [ ]:
class EloModel:
    """
    ELO Rating System for Hockey Game Prediction.
    
    Supports contextual adjustments for home advantage, rest,
    travel fatigue, injuries, and margin of victory.
    """
    
    def __init__(self, params):
        """
        Initialize model with hyperparameters.
        
        Parameters
        ----------
        params : dict
            k_factor : float
                Rating change rate (typical: 20-40)
            home_advantage : float
                Home ice boost in rating points (typical: 50-150)
            initial_rating : float
                Starting rating for all teams (default: 1500)
            mov_multiplier : float
                Margin of victory weight (0 = disabled)
            mov_method : str
                'linear' or 'logarithmic' scaling for margin
            rest_advantage_per_day : float
                Rating boost per day of rest differential
            b2b_penalty : float
                Penalty for back-to-back games (rest <= 1 day)
        """
        self.params = params
        self.ratings = {}
        self.rating_history = []
    
    def initialize_ratings(self, teams, divisions=None):
        """Initialize team ratings, optionally by division tier."""
        initial = self.params.get('initial_rating', 1500)
        division_ratings = {
            'D1': initial + 100,
            'D2': initial,
            'D3': initial - 100
        }
        
        for i, team in enumerate(teams):
            if divisions is not None and i < len(divisions):
                div = divisions.iloc[i] if hasattr(divisions, 'iloc') else divisions[i]
                self.ratings[team] = division_ratings.get(div, initial)
            else:
                self.ratings[team] = initial
    
    def calculate_expected_score(self, team_elo, opponent_elo):
        """Calculate expected win probability using ELO formula."""
        return 1 / (1 + 10 ** ((opponent_elo - team_elo) / 400))
    
    def calculate_mov_multiplier(self, goal_diff):
        """Calculate margin of victory multiplier."""
        mov = self.params.get('mov_multiplier', 0)
        if mov == 0:
            return 1.0
        
        if self.params.get('mov_method', 'logarithmic') == 'linear':
            return 1 + (abs(goal_diff) * mov)
        return 1 + (np.log(abs(goal_diff) + 1) * mov)
    
    def adjust_for_context(self, team_elo, is_home, rest_time, travel_dist, injuries):
        """Apply contextual adjustments to ELO rating."""
        adjusted = team_elo
        
        if is_home:
            adjusted += self.params.get('home_advantage', 0)
        
        if rest_time <= 1:
            adjusted -= self.params.get('b2b_penalty', 0)
        
        if not is_home and travel_dist > 0:
            adjusted -= (travel_dist / 1000) * 15
        
        adjusted -= injuries * 25
        
        return adjusted
    
    def update_ratings(self, game):
        """Update team ratings after a game."""
        home_team = game['home_team']
        away_team = game['away_team']
        
        home_elo = self.ratings.get(home_team, 1500)
        away_elo = self.ratings.get(away_team, 1500)
        
        # Get context values with defaults
        home_rest = game.get('home_rest', 2)
        away_rest = game.get('away_rest', 2)
        away_travel = game.get('away_travel_dist', game.get('travel_distance', 0))
        home_injuries = game.get('home_injuries', 0)
        away_injuries = game.get('away_injuries', 0)
        
        # Apply contextual adjustments
        home_adj = self.adjust_for_context(home_elo, True, home_rest, 0, home_injuries)
        away_adj = self.adjust_for_context(away_elo, False, away_rest, away_travel, away_injuries)
        
        # Rest differential advantage
        rest_diff = home_rest - away_rest
        home_adj += rest_diff * self.params.get('rest_advantage_per_day', 0)
        
        # Calculate expected score
        home_expected = self.calculate_expected_score(home_adj, away_adj)
        
        # Determine actual outcome
        home_actual = 1.0 if game['home_goals'] > game['away_goals'] else 0.0
        
        # Calculate margin of victory multiplier
        goal_diff = game['home_goals'] - game['away_goals']
        mov_mult = self.calculate_mov_multiplier(goal_diff)
        
        # Update ratings
        k = self.params.get('k_factor', 32) * mov_mult
        self.ratings[home_team] = home_elo + k * (home_actual - home_expected)
        self.ratings[away_team] = away_elo + k * ((1 - home_actual) - (1 - home_expected))
        
        # Store history
        self.rating_history.append({
            'home_team': home_team,
            'away_team': away_team,
            'home_rating': self.ratings[home_team],
            'away_rating': self.ratings[away_team]
        })
    
    def predict_goals(self, game):
        """Predict goals for both teams."""
        home_team = game['home_team']
        away_team = game['away_team']
        
        home_elo = self.ratings.get(home_team, 1500)
        away_elo = self.ratings.get(away_team, 1500)
        
        home_rest = game.get('home_rest', 2)
        away_rest = game.get('away_rest', 2)
        away_travel = game.get('away_travel_dist', game.get('travel_distance', 0))
        home_injuries = game.get('home_injuries', 0)
        away_injuries = game.get('away_injuries', 0)
        
        home_adj = self.adjust_for_context(home_elo, True, home_rest, 0, home_injuries)
        away_adj = self.adjust_for_context(away_elo, False, away_rest, away_travel, away_injuries)
        
        rest_diff = home_rest - away_rest
        home_adj += rest_diff * self.params.get('rest_advantage_per_day', 0)
        
        home_win_prob = self.calculate_expected_score(home_adj, away_adj)
        expected_diff = (home_win_prob - 0.5) * 12
        
        home_goals = 3.0 + (expected_diff / 2)
        away_goals = 3.0 - (expected_diff / 2)
        
        return home_goals, away_goals
    
    def fit(self, games_df):
        """Train the model on historical games."""
        teams = pd.concat([games_df['home_team'], games_df['away_team']]).unique()
        
        if 'division' in games_df.columns:
            divisions = games_df.groupby('home_team')['division'].first()
            self.initialize_ratings(teams, divisions)
        else:
            self.initialize_ratings(teams)
        
        for _, game in games_df.iterrows():
            self.update_ratings(game)
    
    def evaluate(self, games_df):
        """Evaluate model on test set."""
        predictions, actuals = [], []
        
        for _, game in games_df.iterrows():
            home_pred, _ = self.predict_goals(game)
            predictions.append(home_pred)
            actuals.append(game['home_goals'])
        
        rmse = mean_squared_error(actuals, predictions, squared=False)
        mae = mean_absolute_error(actuals, predictions)
        r2 = r2_score(actuals, predictions) if len(set(actuals)) > 1 else 0.0
        
        return {'rmse': rmse, 'mae': mae, 'r2': r2}
    
    def get_rankings(self, top_n=None):
        """Get team rankings sorted by ELO rating."""
        sorted_ratings = sorted(self.ratings.items(), key=lambda x: x[1], reverse=True)
        return sorted_ratings[:top_n] if top_n else sorted_ratings

print("EloModel class defined successfully.")

## 3. Loading and Preparing Data

The model expects a DataFrame with the following columns:

| Column | Type | Description |
|--------|------|-------------|
| home_team | str | Home team identifier |
| away_team | str | Away team identifier |
| home_goals | int | Goals scored by home team |
| away_goals | int | Goals scored by away team |
| game_date | date | Game date (for chronological ordering) |
| home_rest | int | Days since home team's last game (optional) |
| away_rest | int | Days since away team's last game (optional) |
| travel_distance | float | Away team travel distance in miles (optional) |
| division | str | Team division: D1, D2, or D3 (optional) |

In [ ]:
# Create sample data for demonstration
# In practice, replace this with: games_df = pd.read_csv('path/to/hockey_data.csv')

sample_data = {
    'game_date': pd.date_range('2025-10-01', periods=20, freq='2D'),
    'home_team': ['Team A', 'Team B', 'Team C', 'Team D', 'Team A',
                  'Team B', 'Team C', 'Team D', 'Team A', 'Team B',
                  'Team C', 'Team D', 'Team A', 'Team B', 'Team C',
                  'Team D', 'Team A', 'Team B', 'Team C', 'Team D'],
    'away_team': ['Team B', 'Team C', 'Team D', 'Team A', 'Team C',
                  'Team D', 'Team A', 'Team B', 'Team D', 'Team A',
                  'Team B', 'Team C', 'Team B', 'Team C', 'Team D',
                  'Team A', 'Team C', 'Team D', 'Team A', 'Team B'],
    'home_goals': [3, 2, 4, 1, 5, 2, 3, 4, 2, 3, 4, 1, 3, 2, 5, 2, 4, 3, 2, 3],
    'away_goals': [2, 3, 1, 4, 2, 3, 2, 2, 4, 1, 2, 3, 1, 4, 2, 3, 1, 2, 3, 2],
    'home_rest': [3, 2, 4, 3, 2, 3, 2, 4, 3, 2, 3, 2, 4, 3, 2, 3, 2, 4, 3, 2],
    'away_rest': [2, 3, 2, 4, 3, 2, 3, 2, 2, 3, 2, 4, 3, 2, 3, 2, 3, 2, 4, 3],
    'division': ['D1', 'D1', 'D2', 'D2', 'D1', 'D1', 'D2', 'D2', 'D1', 'D1',
                 'D2', 'D2', 'D1', 'D1', 'D2', 'D2', 'D1', 'D1', 'D2', 'D2']
}

games_df = pd.DataFrame(sample_data)
games_df = games_df.sort_values('game_date').reset_index(drop=True)

print(f"Dataset: {len(games_df)} games")
print(f"Teams: {games_df['home_team'].nunique()}")
print(f"Date range: {games_df['game_date'].min()} to {games_df['game_date'].max()}")
print("\nFirst 5 games:")
games_df.head()

## 4. Training the Model

Training involves:
1. Splitting data chronologically (80/20 train/test)
2. Initializing team ratings
3. Processing games in order, updating ratings after each

In [ ]:
# Split data chronologically
split_idx = int(len(games_df) * 0.8)
train_df = games_df[:split_idx]
test_df = games_df[split_idx:]

print(f"Training set: {len(train_df)} games")
print(f"Test set: {len(test_df)} games")

In [ ]:
# Define hyperparameters
params = {
    'k_factor': 32,              # Rating change rate
    'home_advantage': 100,       # Home ice boost (rating points)
    'initial_rating': 1500,      # Starting rating
    'mov_multiplier': 1.0,       # Margin of victory weight
    'mov_method': 'logarithmic', # MOV scaling method
    'rest_advantage_per_day': 10,# Rest differential bonus
    'b2b_penalty': 50            # Back-to-back penalty
}

# Initialize and train model
model = EloModel(params)
model.fit(train_df)

print("Model trained successfully.")
print(f"\nTeam ratings after training:")
for team, rating in model.get_rankings():
    print(f"  {team}: {rating:.1f}")

## 5. Evaluating Performance

We evaluate using standard regression metrics:
- **RMSE**: Root Mean Squared Error (lower is better)
- **MAE**: Mean Absolute Error (lower is better)
- **R-squared**: Proportion of variance explained (higher is better)

In [ ]:
# Evaluate on test set
metrics = model.evaluate(test_df)

print("Test Set Performance:")
print(f"  RMSE: {metrics['rmse']:.3f}")
print(f"  MAE:  {metrics['mae']:.3f}")
print(f"  R2:   {metrics['r2']:.3f}")

In [ ]:
# Visualize rating progression
history_df = pd.DataFrame(model.rating_history)

fig, ax = plt.subplots(figsize=(10, 6))

for team in games_df['home_team'].unique():
    team_history = history_df[history_df['home_team'] == team]['home_rating']
    if len(team_history) > 0:
        ax.plot(team_history.values, label=team, linewidth=2)

ax.set_xlabel('Game Number')
ax.set_ylabel('ELO Rating')
ax.set_title('Team Rating Progression Over Time')
ax.legend(loc='best')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 6. Making Predictions

Once trained, the model can predict:
1. Expected goals for each team
2. Win probability based on rating difference

In [ ]:
# Predict a single game
upcoming_game = {
    'home_team': 'Team A',
    'away_team': 'Team B',
    'home_rest': 3,
    'away_rest': 1,  # Team B on back-to-back
    'travel_distance': 500
}

home_pred, away_pred = model.predict_goals(upcoming_game)

print("Prediction for upcoming game:")
print(f"  {upcoming_game['home_team']} vs {upcoming_game['away_team']}")
print(f"  Expected score: {home_pred:.1f} - {away_pred:.1f}")

# Calculate win probability
home_rating = model.ratings.get('Team A', 1500)
away_rating = model.ratings.get('Team B', 1500)
win_prob = model.calculate_expected_score(home_rating + 100, away_rating - 50)  # With adjustments
print(f"  Home win probability: {win_prob:.1%}")

In [ ]:
# Generate predictions for multiple games
def generate_predictions(model, games):
    """Generate predictions for a set of games."""
    results = []
    for _, game in games.iterrows():
        home_pred, away_pred = model.predict_goals(game)
        results.append({
            'home_team': game['home_team'],
            'away_team': game['away_team'],
            'home_goals_pred': round(home_pred, 2),
            'away_goals_pred': round(away_pred, 2),
            'home_goals_actual': game.get('home_goals'),
            'away_goals_actual': game.get('away_goals')
        })
    return pd.DataFrame(results)

predictions = generate_predictions(model, test_df)
print("Test Set Predictions:")
predictions

## 7. Hyperparameter Tuning

The model performance depends on hyperparameter choices. Key parameters to tune:

| Parameter | Range | Description |
|-----------|-------|-------------|
| k_factor | 20-40 | Higher = more volatile ratings |
| home_advantage | 50-150 | Higher = stronger home benefit |
| mov_multiplier | 0-1.5 | Higher = blowouts matter more |
| rest_advantage_per_day | 0-15 | Higher = rest matters more |
| b2b_penalty | 0-75 | Higher = fatigue matters more |

In [ ]:
# Simple grid search example
from itertools import product

# Define parameter grid
param_grid = {
    'k_factor': [20, 32, 40],
    'home_advantage': [50, 100, 150],
    'mov_multiplier': [0, 1.0],
    'rest_advantage_per_day': [0, 10],
    'b2b_penalty': [0, 50]
}

# Generate all combinations
keys = param_grid.keys()
combinations = list(product(*param_grid.values()))

print(f"Total configurations to test: {len(combinations)}")

# Run grid search
results = []
for combo in combinations:
    params = dict(zip(keys, combo))
    params['initial_rating'] = 1500
    params['mov_method'] = 'logarithmic'
    
    model = EloModel(params)
    model.fit(train_df)
    metrics = model.evaluate(test_df)
    
    results.append({
        **params,
        'rmse': metrics['rmse'],
        'mae': metrics['mae'],
        'r2': metrics['r2']
    })

results_df = pd.DataFrame(results)
print("\nGrid search complete.")

In [ ]:
# Find best configuration
best_idx = results_df['rmse'].idxmin()
best_config = results_df.loc[best_idx]

print("Best Configuration:")
print(f"  RMSE: {best_config['rmse']:.3f}")
print(f"  MAE:  {best_config['mae']:.3f}")
print(f"  R2:   {best_config['r2']:.3f}")
print("\nParameters:")
for key in param_grid.keys():
    print(f"  {key}: {best_config[key]}")

In [ ]:
# Visualize hyperparameter impact
fig, axes = plt.subplots(2, 3, figsize=(14, 8))

param_names = ['k_factor', 'home_advantage', 'mov_multiplier', 
               'rest_advantage_per_day', 'b2b_penalty']

for ax, param in zip(axes.flatten()[:5], param_names):
    grouped = results_df.groupby(param)['rmse'].mean()
    ax.bar(range(len(grouped)), grouped.values, color='steelblue')
    ax.set_xticks(range(len(grouped)))
    ax.set_xticklabels([str(x) for x in grouped.index])
    ax.set_xlabel(param)
    ax.set_ylabel('Mean RMSE')
    ax.set_title(f'Impact of {param}')

axes[1, 2].axis('off')  # Hide empty subplot
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:

1. **ELO Model Setup**: Initializing the model with configurable hyperparameters
2. **Data Preparation**: Required columns and chronological ordering
3. **Training**: Fitting the model on historical game data
4. **Evaluation**: Measuring performance with RMSE, MAE, and R-squared
5. **Prediction**: Generating goal predictions for new games
6. **Tuning**: Grid search for optimal hyperparameters

### Next Steps

- Use the full training notebook (`training/train_elo.ipynb`) for comprehensive hyperparameter search
- Run validation tests (`validation/validate_elo.ipynb`) to verify model behavior
- Import the reusable module: `from utils.elo_model import EloModel`